In [760]:
#import thư viện
import numpy as np
from math import nan, isnan
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler , RobustScaler, StandardScaler
import warnings
import re
import time
import csv
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from geopy.geocoders import Nominatim
warnings.filterwarnings("ignore")

Đọc dataBatDongSanDN

In [761]:
dataBatDongSanDN = pd.read_csv('sample_data/dataBatDongSanDN.csv',encoding='utf-8')
dataBatDongSanDN

,name,Số phòng ngủ,Pháp lý,Hướng ban công,Hướng nhà,Address,Nội thất,Mặt tiền,Diện tích,Đường vào,Số toilet,Mức giá,Số tầng
0,Chính chủ cần nhượng lại căn villa mặt tiền đư...,4 phòng,Sổ đỏ/ Sổ hồng,Nam,Nam,https://www.google.com/maps/embed/v1/place?q=1...,Đầy đủ,NaN,271 m²,NaN,4 phòng,45 tỷ,3 tầng
1,"Chính chủ cần bán đất tại phường Thọ Quang, qu...",NaN,Sổ đỏ/ Sổ hồng,NaN,Đông - Nam,https://www.google.com/maps/embed/v1/place?q=1...,NaN,4.5 m,90 m²,NaN,NaN,3.5 tỷ,NaN
2,Bán đất dự án 100 m2 thuộc Sơn Trà mặt biển Mỹ...,NaN,Sổ đỏ/ Sổ hồng,NaN,Nam,NaN,NaN,5 m,100.5 m²,NaN,NaN,4 tỷ,NaN
3,"Chính chủ, bán nhà TTTP Đà Nẵng, full nội thất...",4 phòng,Sổ đỏ/ Sổ hồng,Tây - Nam,Tây - Nam,https://www.google.com/maps/embed/v1/place?q=1...,Đầy đủ,5 m,105 m²,7.5 m,4 phòng,6.75 tỷ,3 tầng
4,Hàng hiếm lô góc kinh doanh Phùng Hưng gần biể...,NaN,Sổ đỏ/ Sổ hồng,NaN,Tây - Bắc,https://www.google.com/maps/embed/v1/place?q=1...,NaN,11.4 m,225 m²,10.5 m,NaN,13 tỷ,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6354,Bán nhà cấp 4 mặt tiền đường 7.5m đường Nguyễn...,2 phòng,NaN,NaN,NaN,https://www.google.com/maps/embed/v1/place?q=1...,NaN,NaN,86.86 m²,7.5 m,NaN,5.6 tỷ,NaN
6355,"Cần bán gấp nhà kiệt Lê Cơ, Phường Hòa Cường B...",4 phòng,NaN,NaN,NaN,https://www.google.com/maps/embed/v1/place?q=1...,NaN,NaN,58.9 m²,NaN,3 phòng,3.1 tỷ,3 tầng
6356,"Cần bán đất Cao Hồng Lãnh, phường Hòa Xuân, Cẩ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,100 m²,5 m,NaN,3.2 tỷ,NaN
6357,"Bán nhà 99, đường Đỗ Đức Dục, Phường An Khê, Q...",3 phòng,NaN,NaN,NaN,NaN,NaN,NaN,85 m²,5.5 m,NaN,4.8 tỷ,NaN


Xem các thông tin cơ bản của dataset dataBatDongSanDN

In [762]:
dataBatDongSanDN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            6359 non-null   object
 1   Số phòng ngủ    2748 non-null   object
 2   Pháp lý         4221 non-null   object
 3   Hướng ban công  1058 non-null   object
 4   Hướng nhà       3827 non-null   object
 5   Address         5905 non-null   object
 6   Nội thất        1667 non-null   object
 7   Mặt tiền        3345 non-null   object
 8   Diện tích       6358 non-null   object
 9   Đường vào       3459 non-null   object
 10  Số toilet       2478 non-null   object
 11  Mức giá         6358 non-null   object
 12  Số tầng         2165 non-null   object
dtypes: object(13)
memory usage: 646.0+ KB


Xóa cột Nội Thất, Hướng ban công và Số Toilet để đồng bộ với dataAloNhaDat

In [763]:
dataBatDongSanDN = dataBatDongSanDN.drop(columns=["Số toilet", "Hướng ban công", "Nội thất"])
dataBatDongSanDN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          6359 non-null   object
 1   Số phòng ngủ  2748 non-null   object
 2   Pháp lý       4221 non-null   object
 3   Hướng nhà     3827 non-null   object
 4   Address       5905 non-null   object
 5   Mặt tiền      3345 non-null   object
 6   Diện tích     6358 non-null   object
 7   Đường vào     3459 non-null   object
 8   Mức giá       6358 non-null   object
 9   Số tầng       2165 non-null   object
dtypes: object(10)
memory usage: 496.9+ KB


Đổi tên các column để đồng bộ và dễ dàng xử lí

In [764]:
dataBatDongSanDN.rename(columns = {
    'Số phòng ngủ':'BedRoooms',
    'Mặt tiền':'Width',
    'Diện tích':'Square',
    'Số tầng':'Floors',
    'Mức giá':'Price',
    'Phòng ăn':'DiningRoom',
    'Pháp lý':'Juridical',
    'Đường vào':'RoadOfFront',
    'Hướng nhà':'Direction',
  }, inplace = True)
dataBatDongSanDN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         6359 non-null   object
 1   BedRoooms    2748 non-null   object
 2   Juridical    4221 non-null   object
 3   Direction    3827 non-null   object
 4   Address      5905 non-null   object
 5   Width        3345 non-null   object
 6   Square       6358 non-null   object
 7   RoadOfFront  3459 non-null   object
 8   Price        6358 non-null   object
 9   Floors       2165 non-null   object
dtypes: object(10)
memory usage: 496.9+ KB


Hàm xem các định dạng sau số

In [765]:
def findAllFormat(data, partern):
  last_rs = []
  temp = list(data)
  for i in temp:
    rs = results_cc = re.findall(partern, str(i))
    for k in rs:
      last_rs.append(k)
  return set(last_rs)

Đưa ColumnsBedRoom về kiểu float

In [766]:
findAllFormat(dataBatDongSanDN['BedRoooms'], "(\D+)")
# result: {' phòng', 'nan'}
def convertToRoom(x):
  pattern_Room = "(\d+|\d+.\d+)(?= phòng)"
  results_Room = re.findall(pattern_Room, x)
  if len(results_Room) > 0:
    return int(results_Room[0])
  return None

dataBatDongSanDN['BedRoooms'] = dataBatDongSanDN['BedRoooms'].apply(lambda x: convertToRoom(x) if x != nan and type(x) == str else nan)
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         6359 non-null   object 
 1   BedRoooms    2748 non-null   float64
 2   Juridical    4221 non-null   object 
 3   Direction    3827 non-null   object 
 4   Address      5905 non-null   object 
 5   Width        3345 non-null   object 
 6   Square       6358 non-null   object 
 7   RoadOfFront  3459 non-null   object 
 8   Price        6358 non-null   object 
 9   Floors       2165 non-null   object 
dtypes: float64(1), object(9)
memory usage: 496.9+ KB


,BedRoooms
count,2748.000000
mean,5.406477
std,13.590760
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,200.000000


Đọc dữ liệu Juridical

In [767]:
dataJuridical = pd.read_csv('sample_data/dataJuridical.csv',encoding='utf-8')
dataJuridical.info()
dataJuridical

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    3 non-null      object
 1   STT     3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 176.0+ bytes


,name,STT
0,Sổ hồng/ Sổ đỏ,1
1,Giấy tờ hợp lệ,2
2,Giấy phép KD,3


Đưa column Juridical về Enum

In [768]:
def converToID(value):
  value = value.lower()
  if value.find('sỗ đỏ') != -1 or value.find('sổ đỏ') != -1 or value.find('sổ hồng') != -1 or value.find('sỗ hồng') != -1 or value.find('có sổ') != -1 or value.find('có sỗ') != -1:
    return dataJuridical.iloc[0]['STT']
  if value.find('gcn qsd') != -1 or value.find('hợp đồng mua bán') != -1:
    return dataJuridical.iloc[1]['STT']
  return None

dataBatDongSanDN['Juridical'] = dataBatDongSanDN['Juridical'].apply(lambda x: converToID(x) if x != nan and type(x) == str else nan)
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         6359 non-null   object 
 1   BedRoooms    2748 non-null   float64
 2   Juridical    4145 non-null   float64
 3   Direction    3827 non-null   object 
 4   Address      5905 non-null   object 
 5   Width        3345 non-null   object 
 6   Square       6358 non-null   object 
 7   RoadOfFront  3459 non-null   object 
 8   Price        6358 non-null   object 
 9   Floors       2165 non-null   object 
dtypes: float64(2), object(8)
memory usage: 496.9+ KB


,BedRoooms,Juridical
count,2748.000000,4145.000000
mean,5.406477,1.021230
std,13.590760,0.144169
min,1.000000,1.000000
25%,2.000000,1.000000
50%,3.000000,1.000000
75%,4.000000,1.000000
max,200.000000,2.000000


Đọc dataDirection

In [769]:
dataDirection = pd.read_csv('sample_data/dataDirection.csv',encoding='utf-8')
dataDirection.info()
dataDirection

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    8 non-null      object
 1   STT     9 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 272.0+ bytes


,name,STT
0,NaN,1
1,Đông,2
2,Bắc,3
3,Tây Bắc,4
4,Đông Nam,5
5,Tây,6
6,Tây Nam,7
7,Đông Bắc,8
8,Nam,9


Đưa column Direction về dạng ID

In [770]:
def converDicrectionToID(value):
  value = value.replace(' -', '')
  resultFind = dataDirection[dataDirection['name'] == value]
  if len(resultFind) > 0:
    return resultFind.iloc[0]['STT']
  return None

dataBatDongSanDN['Direction'] = dataBatDongSanDN['Direction'].apply(lambda x: converDicrectionToID(x) if x != nan and type(x) == str else nan)
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         6359 non-null   object 
 1   BedRoooms    2748 non-null   float64
 2   Juridical    4145 non-null   float64
 3   Direction    3827 non-null   float64
 4   Address      5905 non-null   object 
 5   Width        3345 non-null   object 
 6   Square       6358 non-null   object 
 7   RoadOfFront  3459 non-null   object 
 8   Price        6358 non-null   object 
 9   Floors       2165 non-null   object 
dtypes: float64(3), object(7)
memory usage: 496.9+ KB


,BedRoooms,Juridical,Direction
count,2748.000000,4145.000000,3827.000000
mean,5.406477,1.021230,5.330546
std,13.590760,0.144169,2.231321
min,1.000000,1.000000,2.000000
25%,2.000000,1.000000,4.000000
50%,3.000000,1.000000,5.000000
75%,4.000000,1.000000,7.000000
max,200.000000,2.000000,9.000000


Đưa Square về cùng đơn vị m2 và kiểu Float

In [771]:
findAllFormat(dataBatDongSanDN['Square'], "(\D+)")
# result: {' m²', '.', 'nan'}
def converToMetter(x):
  pattern_Square = "(\d+|\d+.\d+)(?= m²)"
  results_Square = re.findall(pattern_Square, x)
  if len(results_Square) > 0:
    return round(float(results_Square[0]), 3)
  return None

dataBatDongSanDN['Square'] = dataBatDongSanDN['Square'].apply(lambda x: converToMetter(x) if x != nan and type(x) == str else nan)
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         6359 non-null   object 
 1   BedRoooms    2748 non-null   float64
 2   Juridical    4145 non-null   float64
 3   Direction    3827 non-null   float64
 4   Address      5905 non-null   object 
 5   Width        3345 non-null   object 
 6   Square       6358 non-null   float64
 7   RoadOfFront  3459 non-null   object 
 8   Price        6358 non-null   object 
 9   Floors       2165 non-null   object 
dtypes: float64(4), object(6)
memory usage: 496.9+ KB


,BedRoooms,Juridical,Direction,Square
count,2748.000000,4145.000000,3827.000000,6358.000000
mean,5.406477,1.021230,5.330546,869.376206
std,13.590760,0.144169,2.231321,16322.606034
min,1.000000,1.000000,2.000000,14.200000
25%,2.000000,1.000000,4.000000,84.050000
50%,3.000000,1.000000,5.000000,108.000000
75%,4.000000,1.000000,7.000000,200.000000
max,200.000000,2.000000,9.000000,800000.000000


Đưa column Width về chung đơn vị m và kiểu float

In [772]:
findAllFormat(dataBatDongSanDN['Width'], "(\D+)")
# result: {' m', ',', '.', 'nan'}

def converWidthToMetter(x):
  x = x.replace(',', '.')
  pattern_Square = "(\d+|\d+.\d+)(?= m)"
  results_Square = re.findall(pattern_Square, x)
  if len(results_Square) > 0:
    return round(float(results_Square[0]), 3)
  return None

dataBatDongSanDN['Width'] = dataBatDongSanDN['Width'].apply(lambda x: converWidthToMetter(x) if x != nan and type(x) == str else nan)
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         6359 non-null   object 
 1   BedRoooms    2748 non-null   float64
 2   Juridical    4145 non-null   float64
 3   Direction    3827 non-null   float64
 4   Address      5905 non-null   object 
 5   Width        3345 non-null   float64
 6   Square       6358 non-null   float64
 7   RoadOfFront  3459 non-null   object 
 8   Price        6358 non-null   object 
 9   Floors       2165 non-null   object 
dtypes: float64(5), object(5)
memory usage: 496.9+ KB


,BedRoooms,Juridical,Direction,Width,Square
count,2748.000000,4145.000000,3827.000000,3345.000000,6358.000000
mean,5.406477,1.021230,5.330546,8.985761,869.376206
std,13.590760,0.144169,2.231321,15.304393,16322.606034
min,1.000000,1.000000,2.000000,1.000000,14.200000
25%,2.000000,1.000000,4.000000,5.000000,84.050000
50%,3.000000,1.000000,5.000000,6.000000,108.000000
75%,4.000000,1.000000,7.000000,10.000000,200.000000
max,200.000000,2.000000,9.000000,500.000000,800000.000000


Đưa Column Floors về chung đơn vị và kiểu float

In [773]:
findAllFormat(dataBatDongSanDN['Floors'], "(\D+)")
# result: {' tầng', 'nan'}

def convertToFloors(x):
  x = x.replace(',', '.')
  pattern_Square = "(\d+|\d+.\d+)(?= tầng)"
  results_Square = re.findall(pattern_Square, x)
  if len(results_Square) > 0:
    return int(results_Square[0])
  return None

dataBatDongSanDN['Floors'] = dataBatDongSanDN['Floors'].apply(lambda x: convertToFloors(x) if x != nan and type(x) == str else nan)
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         6359 non-null   object 
 1   BedRoooms    2748 non-null   float64
 2   Juridical    4145 non-null   float64
 3   Direction    3827 non-null   float64
 4   Address      5905 non-null   object 
 5   Width        3345 non-null   float64
 6   Square       6358 non-null   float64
 7   RoadOfFront  3459 non-null   object 
 8   Price        6358 non-null   object 
 9   Floors       2165 non-null   float64
dtypes: float64(6), object(4)
memory usage: 496.9+ KB


,BedRoooms,Juridical,Direction,Width,Square,Floors
count,2748.000000,4145.000000,3827.000000,3345.000000,6358.000000,2165.000000
mean,5.406477,1.021230,5.330546,8.985761,869.376206,3.372286
std,13.590760,0.144169,2.231321,15.304393,16322.606034,2.436622
min,1.000000,1.000000,2.000000,1.000000,14.200000,1.000000
25%,2.000000,1.000000,4.000000,5.000000,84.050000,2.000000
50%,3.000000,1.000000,5.000000,6.000000,108.000000,3.000000
75%,4.000000,1.000000,7.000000,10.000000,200.000000,4.000000
max,200.000000,2.000000,9.000000,500.000000,800000.000000,24.000000


Đưa Column Price về chung đơn vị triệu/m2 và kiểu float

In [774]:
findAllFormat(dataBatDongSanDN['Price'], "(\D+)")
# result: {' nghìn', ' triệu', ' triệu/m²', ' tỷ', '.', 'Thỏa thuận', 'nan'}

def convertMPerMetter(id):
  dataTemp = dataBatDongSanDN[dataBatDongSanDN['PricePerMet'] == id]
  value = str(dataTemp.iloc[0]['Price']).replace("nghìn", "tỷ").replace('16500 tỷ', '16.500 tỷ')
  square = dataTemp.iloc[0]['Square']
  pattern_mPerMetter = "(\d+|\d+.\d+)(?= triệu/m²)"
  results_mPerMetter = re.findall(pattern_mPerMetter, value)
  pattern_Billion = "(\d+|\d+.\d+)(?= tỷ)"
  results_Billion = re.findall(pattern_Billion, value)
  pattern_Million = "(\d+|\d+.\d+)(?= triệu)"
  results_Million = re.findall(pattern_Million, value)
  if len(results_mPerMetter) > 0:
    return round(float(results_mPerMetter[0]), 3)
  if len(results_Billion) > 0:
    priceAll = round(float(results_Billion[0]), 3)
    pricePercen = round((priceAll*1000/square), 2)
    return pricePercen
  if len(results_Million) > 0:
    return round((float(results_Million[0])/square), 2)
  return None

dataBatDongSanDN['PricePerMet'] = dataBatDongSanDN.index + 1
dataBatDongSanDN['PricePerMet'] = dataBatDongSanDN['PricePerMet'].apply(lambda x: convertMPerMetter(x) if not isnan(x) else nan)
dataBatDongSanDN = dataBatDongSanDN.drop(columns=['Price'])
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         6359 non-null   object 
 1   BedRoooms    2748 non-null   float64
 2   Juridical    4145 non-null   float64
 3   Direction    3827 non-null   float64
 4   Address      5905 non-null   object 
 5   Width        3345 non-null   float64
 6   Square       6358 non-null   float64
 7   RoadOfFront  3459 non-null   object 
 8   Floors       2165 non-null   float64
 9   PricePerMet  5386 non-null   float64
dtypes: float64(7), object(3)
memory usage: 496.9+ KB


,BedRoooms,Juridical,Direction,Width,Square,Floors,PricePerMet
count,2748.000000,4145.000000,3827.000000,3345.000000,6358.000000,2165.000000,5386.000000
mean,5.406477,1.021230,5.330546,8.985761,869.376206,3.372286,76.881783
std,13.590760,0.144169,2.231321,15.304393,16322.606034,2.436622,77.441682
min,1.000000,1.000000,2.000000,1.000000,14.200000,1.000000,0.010000
25%,2.000000,1.000000,4.000000,5.000000,84.050000,2.000000,31.700000
50%,3.000000,1.000000,5.000000,6.000000,108.000000,3.000000,55.350000
75%,4.000000,1.000000,7.000000,10.000000,200.000000,4.000000,93.162500
max,200.000000,2.000000,9.000000,500.000000,800000.000000,24.000000,1127.920000


Đọc dataType

In [775]:
dataType = pd.read_csv('sample_data/dataType.csv',encoding='utf-8')
dataType.info()
dataType

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    15 non-null     object
 1   STT     15 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 368.0+ bytes


,name,STT
0,"Đất thổ cư, đất ở",1
1,Các loại khác,2
2,"Nhà hàng, khách sạn",3
3,"Shop, kiot, quán",4
4,Căn hộ chung cư,5
5,"Đất nông, lâm nghiệp",6
6,Trang trại,7
7,Nhà mặt tiền,8
8,"Phòng trọ, nhà trọ",9
9,"Kho, xưởng",10


Tạo column type từ name

In [776]:
def getType(x):
  x = x.lower()
  if x.find('mặt bằng') != -1:
    return 15 
  if x.find('biệt thự') != -1 or x.find('nhà liền kề') != -1 or x.find('nhà vườn') != -1:
    return 14
  if x.find('đất nền') != -1 or x.find('lô liền kề') != -1 or x.find('đất liền kề') != -1 or x.find('đất dự án') != -1:
    return 13
  if x.find('nhà trong hẻm') != -1:
    return 12
  if x.find('văn phòng') != -1:
    return 11
  if x.find('kho') != -1 | x.find('xưởng') != -1:
    return 10
  if x.find('kho') != -1 | x.find('xưởng') != -1:
    return 9
  if x.find('phòng trọ') != -1 | x.find('nhà trọ') != -1:
    return 8
  if x.find('trang trại') != -1:
    return 7
  if x.find('đất nông nghiệp') != -1 or x.find('đất lâm nghiệp') != -1:
    return 6
  if x.find('căn hộ chung cư') != -1 or x.find('căn hộ') != -1:
    return 5
  if x.find('quán') != -1 or x.find('shop') != -1 or x.find('kiot') != -1:
    return 4
  if x.find('khách sạn') != -1 or x.find('nhà hàng') != -1:
    return 3
  if x.find('đất thổ cư') != -1 or x.find('đất ở') != -1:
    return 1
  return 2

dataBatDongSanDN['name'] = dataBatDongSanDN['name'].apply(lambda x: getType(x))
dataBatDongSanDN.rename(columns = {'name':'Type'}, inplace = True)
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Type         6359 non-null   int64  
 1   BedRoooms    2748 non-null   float64
 2   Juridical    4145 non-null   float64
 3   Direction    3827 non-null   float64
 4   Address      5905 non-null   object 
 5   Width        3345 non-null   float64
 6   Square       6358 non-null   float64
 7   RoadOfFront  3459 non-null   object 
 8   Floors       2165 non-null   float64
 9   PricePerMet  5386 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 496.9+ KB


,Type,BedRoooms,Juridical,Direction,Width,Square,Floors,PricePerMet
count,6359.000000,2748.000000,4145.000000,3827.000000,3345.000000,6358.000000,2165.000000,5386.000000
mean,3.662368,5.406477,1.021230,5.330546,8.985761,869.376206,3.372286,76.881783
std,3.746994,13.590760,0.144169,2.231321,15.304393,16322.606034,2.436622,77.441682
min,1.000000,1.000000,1.000000,2.000000,1.000000,14.200000,1.000000,0.010000
25%,2.000000,2.000000,1.000000,4.000000,5.000000,84.050000,2.000000,31.700000
50%,2.000000,3.000000,1.000000,5.000000,6.000000,108.000000,3.000000,55.350000
75%,2.000000,4.000000,1.000000,7.000000,10.000000,200.000000,4.000000,93.162500
max,15.000000,200.000000,2.000000,9.000000,500.000000,800000.000000,24.000000,1127.920000


Đưa column Address về địa chỉ BDS

In [777]:
def getAddressFromLocation(x):
  value = x.split('&')[0]
  valueLocation = value.split('?q=')[1]
  try:
    location = Nominatim(user_agent="GetLoc", timeout = 3)
    getLocation = location.reverse(value)
    return getLocation.address
  except Exception as e: 
    print(e)
    return None

# dataBatDongSanDN['Address'] = dataBatDongSanDN['Address'].apply(lambda x: getAddressFromLocation(x) if type(x) == str else nan)
# dataBatDongSanDN.info()
# dataBatDongSanDN.describe()

In [778]:
dataBatDongSanDN = pd.read_csv('dataBDSAddress.csv',encoding='utf-8')

Hàm xóa các trường ở cuối

In [779]:
def RemoveLastParams(address):
  value = address.split(', ')
  return address.replace(', ' + value[len(value) - 1], "")

Xóa các record không phải ở Đà Nẵng và xóa Thành Phố Đà Nẵng

In [780]:
def removeDN(x):
  if(x.find('Thành phố Đà Nẵng') != -1):
    return x.split(', Thành phố Đà Nẵng')[0]
  else:
    return None

dataBatDongSanDN['Address'] = dataBatDongSanDN['Address'].apply(lambda x: removeDN(x) if type(x) == str else nan)
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   6359 non-null   int64  
 1   Type         6359 non-null   int64  
 2   BedRoooms    2748 non-null   float64
 3   Juridical    4145 non-null   float64
 4   Direction    3827 non-null   float64
 5   Address      5850 non-null   object 
 6   Width        3345 non-null   float64
 7   Square       6358 non-null   float64
 8   RoadOfFront  3459 non-null   object 
 9   Floors       2165 non-null   float64
 10  PricePerMet  5386 non-null   float64
dtypes: float64(7), int64(2), object(2)
memory usage: 546.6+ KB


,Unnamed: 0,Type,BedRoooms,Juridical,Direction,Width,Square,Floors,PricePerMet
count,6359.000000,6359.000000,2748.000000,4145.000000,3827.000000,3345.000000,6358.000000,2165.000000,5386.000000
mean,3179.000000,3.662368,5.406477,1.021230,5.330546,8.985761,869.376206,3.372286,76.881783
std,1835.829513,3.746994,13.590760,0.144169,2.231321,15.304393,16322.606034,2.436622,77.441682
min,0.000000,1.000000,1.000000,1.000000,2.000000,1.000000,14.200000,1.000000,0.010000
25%,1589.500000,2.000000,2.000000,1.000000,4.000000,5.000000,84.050000,2.000000,31.700000
50%,3179.000000,2.000000,3.000000,1.000000,5.000000,6.000000,108.000000,3.000000,55.350000
75%,4768.500000,2.000000,4.000000,1.000000,7.000000,10.000000,200.000000,4.000000,93.162500
max,6358.000000,15.000000,200.000000,2.000000,9.000000,500.000000,800000.000000,24.000000,1127.920000


Tạo dữ liệu về quận Đà Nẵng

In [781]:
dataDistrictDict = []
dataDistrictDict.append({'Name': 'Quận Hải Châu', 'Point': 1})
dataDistrictDict.append({'Name': 'Quận Liên Chiểu', 'Point': 2})
dataDistrictDict.append({'Name': 'Quận Thanh Khê', 'Point': 3})
dataDistrictDict.append({'Name': 'Quận Cẩm Lệ', 'Point': 4})
dataDistrictDict.append({'Name': 'Quận Sơn Trà', 'Point': 5})
dataDistrictDict.append({'Name': 'Quận Ngũ Hành Sơn', 'Point': 6})
dataDistrictDict.append({'Name': 'Huyện Hòa Vang', 'Point': 7})
dataDistrictDict.append({'Name': 'Huyện Hoàng Sa', 'Point': 8})

dataDistrict = pd.DataFrame.from_dict(dataDistrictDict)
dataDistrict

,Name,Point
0,Quận Hải Châu,1
1,Quận Liên Chiểu,2
2,Quận Thanh Khê,3
3,Quận Cẩm Lệ,4
4,Quận Sơn Trà,5
5,Quận Ngũ Hành Sơn,6
6,Huyện Hòa Vang,7
7,Huyện Hoàng Sa,8


Tạo ra column DistrictPoint cho data

In [782]:
def getPoint(address, sourcePoint):
  value = address.split(", ")
  resultFind = sourcePoint[sourcePoint['Name'].str.find(value[len(value) - 1]) != -1]
  if(len(resultFind) > 0):
    return resultFind.iloc[0]['Point']
  else:
    max_value = sourcePoint['Point'].max()
    global dataDistrictDict
    dataDistrictDict = sourcePoint.append({'Name': value[len(value) - 1], 'Point': max_value + 1},ignore_index=True, verify_integrity=False, sort=None)
    return max_value + 1

dataBatDongSanDN['DistrictPoint'] = dataBatDongSanDN['Address']
dataBatDongSanDN['DistrictPoint'] = dataBatDongSanDN['DistrictPoint'].apply(lambda x: getPoint(x, dataDistrict) if type(x) == str else x)
#Xóa điểm cuối cùng của dữ liệu
dataBatDongSanDN['Address'] = dataBatDongSanDN['Address'].apply(lambda x: RemoveLastParams(x) if type(x) == str else x)
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     6359 non-null   int64  
 1   Type           6359 non-null   int64  
 2   BedRoooms      2748 non-null   float64
 3   Juridical      4145 non-null   float64
 4   Direction      3827 non-null   float64
 5   Address        5850 non-null   object 
 6   Width          3345 non-null   float64
 7   Square         6358 non-null   float64
 8   RoadOfFront    3459 non-null   object 
 9   Floors         2165 non-null   float64
 10  PricePerMet    5386 non-null   float64
 11  DistrictPoint  5850 non-null   float64
dtypes: float64(8), int64(2), object(2)
memory usage: 596.3+ KB


,Unnamed: 0,Type,BedRoooms,Juridical,Direction,Width,Square,Floors,PricePerMet,DistrictPoint
count,6359.000000,6359.000000,2748.000000,4145.000000,3827.000000,3345.000000,6358.000000,2165.000000,5386.000000,5850.000000
mean,3179.000000,3.662368,5.406477,1.021230,5.330546,8.985761,869.376206,3.372286,76.881783,3.988547
std,1835.829513,3.746994,13.590760,0.144169,2.231321,15.304393,16322.606034,2.436622,77.441682,2.026337
min,0.000000,1.000000,1.000000,1.000000,2.000000,1.000000,14.200000,1.000000,0.010000,1.000000
25%,1589.500000,2.000000,2.000000,1.000000,4.000000,5.000000,84.050000,2.000000,31.700000,2.000000
50%,3179.000000,2.000000,3.000000,1.000000,5.000000,6.000000,108.000000,3.000000,55.350000,4.000000
75%,4768.500000,2.000000,4.000000,1.000000,7.000000,10.000000,200.000000,4.000000,93.162500,6.000000
max,6358.000000,15.000000,200.000000,2.000000,9.000000,500.000000,800000.000000,24.000000,1127.920000,7.000000


Đọc data WardDistrict

In [783]:
dataWardDistrict = pd.read_excel('sample_data/WardDistrict.xlsx')
dataWardDistrict.rename(columns = {
    'Phường/Xã':'Name',
    'Quận/Huyện':'DistrictPoint',
  }, inplace = True)
dataWardDistrict['Point'] = dataWardDistrict.index + 1 
dataWardDistrict = dataWardDistrict.drop(columns=['DistrictPoint'])
dataWardDistrict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    57 non-null     object
 1   Point   57 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ KB


Tạo columns WardPoint và get từ data

In [784]:
def getWardPoint(address, sourcePoint):
  value = ""
  if(address.find('Phường') != -1):
    value = address.replace(address.split('Phường')[0], "")
  elif(address.find('Xã') != -1):
    value = address.replace(address.split('Xã')[0], "")
  else:
    return None
  value = value.replace('Phường Chính Gián, 12, Đường Nguyễn Tri Phương, Phường Chính Gián', 'Phường Chính Gián')
  resultFind = sourcePoint[sourcePoint['Name'] == value]
  if(len(resultFind) > 0):
    return resultFind.iloc[0]['Point']
  else:
    max_value = sourcePoint['Point'].max()
    global dataWardDistrict 
    dataWardDistrict = sourcePoint.append({'Name': value, 'Point': max_value + 1},ignore_index=True)
    return max_value + 1

dataBatDongSanDN['WardPoint'] = dataBatDongSanDN['Address']
dataBatDongSanDN['WardPoint'] = dataBatDongSanDN['WardPoint'].apply(lambda x: getWardPoint(x, dataWardDistrict) if type(x) == str else x)
#Xóa điểm cuối cùng của dữ liệu
dataBatDongSanDN['Address'] = dataBatDongSanDN['Address'].apply(lambda x: RemoveLastParams(x) if type(x) == str else x)
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     6359 non-null   int64  
 1   Type           6359 non-null   int64  
 2   BedRoooms      2748 non-null   float64
 3   Juridical      4145 non-null   float64
 4   Direction      3827 non-null   float64
 5   Address        5850 non-null   object 
 6   Width          3345 non-null   float64
 7   Square         6358 non-null   float64
 8   RoadOfFront    3459 non-null   object 
 9   Floors         2165 non-null   float64
 10  PricePerMet    5386 non-null   float64
 11  DistrictPoint  5850 non-null   float64
 12  WardPoint      5581 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 646.0+ KB


,Unnamed: 0,Type,BedRoooms,Juridical,Direction,Width,Square,Floors,PricePerMet,DistrictPoint,WardPoint
count,6359.000000,6359.000000,2748.000000,4145.000000,3827.000000,3345.000000,6358.000000,2165.000000,5386.000000,5850.000000,5581.000000
mean,3179.000000,3.662368,5.406477,1.021230,5.330546,8.985761,869.376206,3.372286,76.881783,3.988547,28.455116
std,1835.829513,3.746994,13.590760,0.144169,2.231321,15.304393,16322.606034,2.436622,77.441682,2.026337,12.756262
min,0.000000,1.000000,1.000000,1.000000,2.000000,1.000000,14.200000,1.000000,0.010000,1.000000,1.000000
25%,1589.500000,2.000000,2.000000,1.000000,4.000000,5.000000,84.050000,2.000000,31.700000,2.000000,19.000000
50%,3179.000000,2.000000,3.000000,1.000000,5.000000,6.000000,108.000000,3.000000,55.350000,4.000000,33.000000
75%,4768.500000,2.000000,4.000000,1.000000,7.000000,10.000000,200.000000,4.000000,93.162500,6.000000,39.000000
max,6358.000000,15.000000,200.000000,2.000000,9.000000,500.000000,800000.000000,24.000000,1127.920000,7.000000,45.000000


Tạo mảng Data Street để ghi dữ liệu Street

In [785]:
dataStreet = pd.read_csv('sample_data/dataStreet.csv')
dataStreet.reset_index(inplace=True, drop=True)
dataStreet.info()
dataStreet.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2279 entries, 0 to 2278
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2279 non-null   int64  
 1   index          1689 non-null   float64
 2   width          1518 non-null   float64
 3   Name           2279 non-null   object 
 4   Point          2279 non-null   int64  
 5   DistrictPoint  1591 non-null   float64
 6   WardPoint      1589 non-null   float64
 7   WidthRoad      1579 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 142.6+ KB


,Unnamed: 0,index,width,Point,DistrictPoint,WardPoint,WidthRoad
count,2279.000000,1689.000000,1518.000000,2279.000000,1591.000000,1589.000000,1579.000000
mean,1139.000000,903.563647,6.742945,1140.000000,3.873664,29.297042,7.234858
std,658.034953,536.391715,3.700123,658.034953,1.727980,13.991132,4.121527
min,0.000000,0.000000,2.500000,1.000000,1.000000,1.000000,1.000000
25%,569.500000,428.000000,5.500000,570.500000,2.000000,19.000000,5.500000
50%,1139.000000,890.000000,5.500000,1140.000000,4.000000,33.000000,6.000000
75%,1708.500000,1369.000000,7.500000,1709.500000,5.000000,40.000000,7.500000
max,2278.000000,1836.000000,56.000000,2279.000000,7.000000,56.000000,56.000000


Gán tên đường bằng Point

In [786]:
def getRoadPoint(address, source):
  value = ""
  if(address.find('Đường') != -1):
    value = address.replace(address.split('Đường')[0], "").replace('Đường ', '').replace('.', '')
    if(value.find('Phường') != -1):
      value = value.replace(value.split(' Phường')[1], '').replace(' Phường', '')
  else:
    return None
  value = value.split(', ')[0]
  resultFind = source[source['Name'].str.lower() == value.lower()]
  if(len(resultFind) > 0):
    return resultFind.iloc[0]['Point']
  else:
    max_value = source['Point'].max()
    global dataStreet 
    dataStreet = source.append({'Name': value, 'Point': max_value + 1},ignore_index=True)
    return max_value + 1

def handleExceptionRoad(x):
    value = x.replace("29 Tháng 3", "29/3").replace("Khách sạn Mỹ Khê II", "Võ Nguyên Giáp").replace("An Ngãi", "ĐT 602")
    value = value.replace("Tránh Nam Hải Vân-Túy Loan", "Trường Sơn").replace("Nam Hải Vân-Túy Loan", "Trường Sơn")
    value = value.replace("cao tốc La Sơn - Túy Loan giai đoạn 2", "La Sơn - Túy Loan").replace("3 Tháng 2", "3/2")
    return value.replace("Đống Công Trường", "Đống Công Tường").replace("30 Tháng 4", "30/4").replace("2 Tháng 9", "2/9")

dataBatDongSanDN['RoadPoint'] = dataBatDongSanDN['Address'].apply(lambda x: handleExceptionRoad(x) if type(x) == str else x)
dataBatDongSanDN['RoadPoint'] = dataBatDongSanDN['RoadPoint'].apply(lambda x: getRoadPoint(x, dataStreet) if type(x) == str else x)
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     6359 non-null   int64  
 1   Type           6359 non-null   int64  
 2   BedRoooms      2748 non-null   float64
 3   Juridical      4145 non-null   float64
 4   Direction      3827 non-null   float64
 5   Address        5850 non-null   object 
 6   Width          3345 non-null   float64
 7   Square         6358 non-null   float64
 8   RoadOfFront    3459 non-null   object 
 9   Floors         2165 non-null   float64
 10  PricePerMet    5386 non-null   float64
 11  DistrictPoint  5850 non-null   float64
 12  WardPoint      5581 non-null   float64
 13  RoadPoint      4260 non-null   float64
dtypes: float64(10), int64(2), object(2)
memory usage: 695.6+ KB


,Unnamed: 0,Type,BedRoooms,Juridical,Direction,Width,Square,Floors,PricePerMet,DistrictPoint,WardPoint,RoadPoint
count,6359.000000,6359.000000,2748.000000,4145.000000,3827.000000,3345.000000,6358.000000,2165.000000,5386.000000,5850.000000,5581.000000,4260.000000
mean,3179.000000,3.662368,5.406477,1.021230,5.330546,8.985761,869.376206,3.372286,76.881783,3.988547,28.455116,1191.361268
std,1835.829513,3.746994,13.590760,0.144169,2.231321,15.304393,16322.606034,2.436622,77.441682,2.026337,12.756262,625.569507
min,0.000000,1.000000,1.000000,1.000000,2.000000,1.000000,14.200000,1.000000,0.010000,1.000000,1.000000,1.000000
25%,1589.500000,2.000000,2.000000,1.000000,4.000000,5.000000,84.050000,2.000000,31.700000,2.000000,19.000000,656.750000
50%,3179.000000,2.000000,3.000000,1.000000,5.000000,6.000000,108.000000,3.000000,55.350000,4.000000,33.000000,1152.000000
75%,4768.500000,2.000000,4.000000,1.000000,7.000000,10.000000,200.000000,4.000000,93.162500,6.000000,39.000000,1692.000000
max,6358.000000,15.000000,200.000000,2.000000,9.000000,500.000000,800000.000000,24.000000,1127.920000,7.000000,45.000000,2347.000000


Xem các record Street được add

In [787]:
dataStreetTemp = dataStreet[(dataStreet['Point'] > 2278 )]
dataStreetTemp

,Unnamed: 0,index,width,Name,Point,DistrictPoint,WardPoint,WidthRoad
2278,2278.0,NaN,NaN,Xuân Thiều 24,2279,2.0,2.0,7.5
2279,NaN,NaN,NaN,Vũng Thùng 6,2280,NaN,NaN,NaN
2280,NaN,NaN,NaN,Số 7,2281,NaN,NaN,NaN
2281,NaN,NaN,NaN,Nguyễn Thành Hân,2282,NaN,NaN,NaN
2282,NaN,NaN,NaN,Hưng Hóa 5,2283,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2342,NaN,NaN,NaN,Tân Hòa 1,2343,NaN,NaN,NaN
2343,NaN,NaN,NaN,An Trung Đông 7,2344,NaN,NaN,NaN
2344,NaN,NaN,NaN,Cao Bá Đạt,2345,NaN,NaN,NaN
2345,NaN,NaN,NaN,Thạch Sơn 2,2346,NaN,NaN,NaN


Get các value width từ dataStreet

In [788]:
def getStreetWidth(id):
  result = nan
  dataTemp = dataBatDongSanDN[dataBatDongSanDN['WidthRoad'] == id]
  roadPoint = dataTemp.iloc[0]['RoadPoint']
  resultFind = dataStreet[dataStreet['Point'] == roadPoint]
  if len(resultFind) > 0:
    result = resultFind.iloc[0]['WidthRoad']
    if not isnan(result):
      return result
    else:
      result = dataTemp.iloc[0]['RoadOfFront']
  else:
    result = dataTemp.iloc[0]['RoadOfFront']
  return result

dataBatDongSanDN['WidthRoad'] = dataBatDongSanDN.index + 1
dataBatDongSanDN['WidthRoad'] = dataBatDongSanDN['WidthRoad'].apply(lambda x: getStreetWidth(x))
# dataBatDongSanDN = dataBatDongSanDN.drop(columns=['RoadOfFront'])
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     6359 non-null   int64  
 1   Type           6359 non-null   int64  
 2   BedRoooms      2748 non-null   float64
 3   Juridical      4145 non-null   float64
 4   Direction      3827 non-null   float64
 5   Address        5850 non-null   object 
 6   Width          3345 non-null   float64
 7   Square         6358 non-null   float64
 8   RoadOfFront    3459 non-null   object 
 9   Floors         2165 non-null   float64
 10  PricePerMet    5386 non-null   float64
 11  DistrictPoint  5850 non-null   float64
 12  WardPoint      5581 non-null   float64
 13  RoadPoint      4260 non-null   float64
 14  WidthRoad      5184 non-null   object 
dtypes: float64(10), int64(2), object(3)
memory usage: 745.3+ KB


,Unnamed: 0,Type,BedRoooms,Juridical,Direction,Width,Square,Floors,PricePerMet,DistrictPoint,WardPoint,RoadPoint
count,6359.000000,6359.000000,2748.000000,4145.000000,3827.000000,3345.000000,6358.000000,2165.000000,5386.000000,5850.000000,5581.000000,4260.000000
mean,3179.000000,3.662368,5.406477,1.021230,5.330546,8.985761,869.376206,3.372286,76.881783,3.988547,28.455116,1191.361268
std,1835.829513,3.746994,13.590760,0.144169,2.231321,15.304393,16322.606034,2.436622,77.441682,2.026337,12.756262,625.569507
min,0.000000,1.000000,1.000000,1.000000,2.000000,1.000000,14.200000,1.000000,0.010000,1.000000,1.000000,1.000000
25%,1589.500000,2.000000,2.000000,1.000000,4.000000,5.000000,84.050000,2.000000,31.700000,2.000000,19.000000,656.750000
50%,3179.000000,2.000000,3.000000,1.000000,5.000000,6.000000,108.000000,3.000000,55.350000,4.000000,33.000000,1152.000000
75%,4768.500000,2.000000,4.000000,1.000000,7.000000,10.000000,200.000000,4.000000,93.162500,6.000000,39.000000,1692.000000
max,6358.000000,15.000000,200.000000,2.000000,9.000000,500.000000,800000.000000,24.000000,1127.920000,7.000000,45.000000,2347.000000


Đưa Column RoadOfFront về chung đơn vị m kiểu dữ liệu float

In [789]:
findAllFormat(dataBatDongSanDN['WidthRoad'], "(\D+)")
# Result: {' m', '.', 'nan'}

def converWitdhToMetter(value):
  pattern_Metter = "(\d+|\d+.\d+)(?= m)"
  result_Metter = re.findall(pattern_Metter, value)
  result = 0
  if len(result_Metter) > 1:
    result = round(float(result_Metter[0].replace(' ', '')) + float(result_Metter[1])/10, 3)
  if len(result_Metter) > 0:
    result = round(float(result_Metter[0].replace(' ', '')), 3)
  while(result > 56):
    result = round(result / 10, 3)
  return result if result > 0 else None

dataBatDongSanDN['WidthRoad'] = dataBatDongSanDN['WidthRoad'].apply(lambda x: converWitdhToMetter(x) if type(x) == str else x)
dataBatDongSanDN = dataBatDongSanDN.drop(columns=['Address', 'RoadOfFront'])
dataBatDongSanDN.info()
dataBatDongSanDN.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     6359 non-null   int64  
 1   Type           6359 non-null   int64  
 2   BedRoooms      2748 non-null   float64
 3   Juridical      4145 non-null   float64
 4   Direction      3827 non-null   float64
 5   Width          3345 non-null   float64
 6   Square         6358 non-null   float64
 7   Floors         2165 non-null   float64
 8   PricePerMet    5386 non-null   float64
 9   DistrictPoint  5850 non-null   float64
 10  WardPoint      5581 non-null   float64
 11  RoadPoint      4260 non-null   float64
 12  WidthRoad      5184 non-null   float64
dtypes: float64(11), int64(2)
memory usage: 646.0 KB


,Unnamed: 0,Type,BedRoooms,Juridical,Direction,Width,Square,Floors,PricePerMet,DistrictPoint,WardPoint,RoadPoint,WidthRoad
count,6359.000000,6359.000000,2748.000000,4145.000000,3827.000000,3345.000000,6358.000000,2165.000000,5386.000000,5850.000000,5581.000000,4260.000000,5184.000000
mean,3179.000000,3.662368,5.406477,1.021230,5.330546,8.985761,869.376206,3.372286,76.881783,3.988547,28.455116,1191.361268,11.606009
std,1835.829513,3.746994,13.590760,0.144169,2.231321,15.304393,16322.606034,2.436622,77.441682,2.026337,12.756262,625.569507,9.681551
min,0.000000,1.000000,1.000000,1.000000,2.000000,1.000000,14.200000,1.000000,0.010000,1.000000,1.000000,1.000000,2.000000
25%,1589.500000,2.000000,2.000000,1.000000,4.000000,5.000000,84.050000,2.000000,31.700000,2.000000,19.000000,656.750000,6.000000
50%,3179.000000,2.000000,3.000000,1.000000,5.000000,6.000000,108.000000,3.000000,55.350000,4.000000,33.000000,1152.000000,7.500000
75%,4768.500000,2.000000,4.000000,1.000000,7.000000,10.000000,200.000000,4.000000,93.162500,6.000000,39.000000,1692.000000,14.000000
max,6358.000000,15.000000,200.000000,2.000000,9.000000,500.000000,800000.000000,24.000000,1127.920000,7.000000,45.000000,2347.000000,56.000000


Lấy ra các trường liên quan đến Street gồm (Street Point, Ward Point, District Point, Width) từ dataAloNhaDat

In [790]:
dataAddressAll = dataBatDongSanDN[['DistrictPoint', 'WardPoint', 'RoadPoint', 'WidthRoad']]
dataAddressAll = dataAddressAll.drop_duplicates(subset=['RoadPoint'])
dataAddressAll = dataAddressAll.dropna(subset=['RoadPoint'])
dataAddressAll = dataAddressAll.sort_values(['RoadPoint'])
dataAddressAll = dataAddressAll.rename(columns = {'RoadPoint': 'Point'})
dataAddressAll.info()
dataAddressAll.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 674 entries, 4692 to 6249
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DistrictPoint  674 non-null    float64
 1   WardPoint      664 non-null    float64
 2   Point          674 non-null    float64
 3   WidthRoad      642 non-null    float64
dtypes: float64(4)
memory usage: 26.3 KB


,DistrictPoint,WardPoint,Point,WidthRoad
count,674.000000,664.000000,674.000000,642.000000
mean,3.777448,28.481928,1265.580119,8.187305
std,1.778448,13.308033,683.088437,5.327030
min,1.000000,1.000000,1.000000,2.500000
25%,2.000000,17.000000,690.000000,5.500000
50%,4.000000,32.000000,1222.500000,7.500000
75%,5.000000,39.000000,1836.500000,8.000000
max,7.000000,45.000000,2347.000000,56.000000


Thêm các cột mới get vào data Street

In [791]:
for i in range(0, len(dataAddressAll)):
  item = dataAddressAll.iloc[i]
  dataStreet.loc[dataStreet[dataStreet['Point'] == item['Point']].index[0],['DistrictPoint', 'WardPoint', 'WidthRoad']] = [item['DistrictPoint'], item['WardPoint'], item['WidthRoad']]

dataStreet = dataStreet.drop(columns=['width'])
dataStreet.info()
dataStreet.describe()
dataStreet.to_csv('dataStreet.csv', encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2347 entries, 0 to 2346
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2279 non-null   float64
 1   index          1689 non-null   float64
 2   Name           2347 non-null   object 
 3   Point          2347 non-null   int64  
 4   DistrictPoint  1701 non-null   float64
 5   WardPoint      1690 non-null   float64
 6   WidthRoad      1662 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 128.5+ KB


Ghi dữ liệu đã làm sạch vào file DataBDSCleaned

In [792]:
dataBatDongSanDN.to_csv('dataBDSCleaned.csv', encoding='utf-8')